<h1> <strong> <span style="color: coral"> Script extracción de contenido 
</strong>
</h1>



In [5]:
#Librerias 
import pandas as pd                              # manejo de Dataframes
import numpy as np                               # manejo de estructuras de datos como arreglos
import json                                      # trabajar con archivos json
import zipfile                                   # work with ZIP files 
from pathlib import Path                         # direccionar archivos
from pathlib import PureWindowsPath, PurePosixPath

import time     
import re
import random
import requests


In [ ]:
#Cargamos Base de Datos 

#Formato xlsx 

df_MS = pd.read_excel("Path/archivo.xlsx")


In [6]:
###### Funciones Web Scraping  #### 

#Diffbot 

#Credenciales
user = 'diegorodolfo.sanchez@alumnos.cide.edu'
API_TOKEN = '69aa088eb4eb88ec567b60a1ba18bce4'

class DiffbotClient(object):

    base_url = 'http://api.diffbot.com/'

    def request(self, url, token, api, fields=None, version=3, **kwargs):
        """
        Returns a python object containing the requested resource from the diffbot api
        """
        params = {"url": url, "token": token}
        if fields:
            params['fields'] = fields
        params.update(kwargs)
        response = requests.get(self.compose_url(api, version), params=params)
        response.raise_for_status()
        return response.json()

    def compose_url(self, api, version_number):
        """
        Returns the uri for an endpoint as a string
        """
        version = self.format_version_string(version_number)
        return '{}{}/{}'.format(self.base_url, version, api)
    @staticmethod
    def format_version_string(version_number):
        """
        Returns a string representation of the API version
        """
        return 'v{}'.format(version_number)


def get_content_diffbot(url):
    diffbot = DiffbotClient()
    token = API_TOKEN
    api = "analyze"
    try:
        response = diffbot.request(url, token, api)
        if 'objects' in response:
            if len(response['objects'])>0:
                if 'text' in response['objects'][0]:
                    #print("URL Content from {} is correct".format(url))
                    return response['objects'][0]['text']
                else:
                    #print("URL Content from {} NOT FOUND".format(url))
                    return "No Content"
            else:
                #print("URL Content from {} NOT FOUND".format(url))
                return "Empty URL, Nothing found"
        else:
            #print("URL Content from {} NOT FOUND".format(url))
            return "Empty URL, Nothing found"
    except:
        return "Something went wrong with url"

#Newspaper 
import newspaper      
"""NOTA: Si no esta instalada abrir Anaconda Prompt y correr lo siguiente: pip install newspaper3k"""
from newspaper import Article, fulltext, Config, ArticleException

#En el siguiente link consiguen su User Agent 

"https://developers.whatismybrowser.com/"  
#agregamos el User a la siguiente lista

user_agent_list = [
   #Chrome
     'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36',

    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 5.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.2; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.90 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/57.0.2987.133 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36',
    #Firefox
    'Mozilla/4.0 (compatible; MSIE 9.0; Windows NT 6.1)',
    'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 6.2; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.0; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.3; WOW64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)',
    'Mozilla/5.0 (Windows NT 6.1; Win64; x64; Trident/7.0; rv:11.0) like Gecko',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; WOW64; Trident/6.0)',
    'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.1; Trident/6.0)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729)'
]

import urllib.request,sys,time

#Function 
def get_content_news(url): 
      user_agent = random.choice(user_agent_list)
      config = Config()


     
      config.browser_user_agent = user_agent
      time.sleep(.5)  

      a  =  Article(url, config=config)

      try:
           a.download()
           a.parse()
           paragraphs = a.text
           paragraphs = re.sub(r'<a href=.+?(?=)>|<br/>|\\xa0|\n|</a>|\xa0|<strong>|</strong>|<br/>•|<i(.*?)</i>|<img(.*?)>','',str(paragraphs))
           if type(paragraphs)==str and  len(paragraphs) >0:
                  print("URL Content from {} is correct".format(url))
                  return paragraphs
         
           #elif type(paragraphs) == newspaper.article.ArticleException:
         
                  #print("URL Content from {} is ArticleException".format(url))
                  #return paragraphs
           elif paragraphs == '' or  type(paragraphs) == newspaper.article.ArticleException:
                 ext_diff= get_content_diffbot(url) 
                 print("URL Content from {} is correct from diffbot".format(url))

                 return ext_diff
      except Exception as exce:
            
             print("URL Content from {} is OtherError".format(url))
             try:
                 ext_diff= get_content_diffbot(url)
                 print("URL Content from {} is correct from diffbot".format(url))

                 return ext_diff
             except: 
                     print("URL Content from {} is OtherError".format(url))





In [9]:
import time     
##Extraemos en cada columna 

inicio = time.time()

for index,row in df_MS.iterrows():   
     
    if row['Source']=='CIMS': ## Si queremos hacer por SUBS el condicional sería : (row['Source']=='CIMS') & (row['Country']=='Nombre de país')

        df_MS.at[index, 'Mention Content'] =  get_content_news(row['Mention URL'])
        print(index)
    elif row['Source']== 'Synthesio': ## Si queremos hacer por SUBS el condicional sería : (row['Source']=='Synthesio') & (row['Country']=='Nombre de país')
        
        content_syn =  get_content_news(row['Mention URL'])

        if (content_syn =="No Content") or (content_syn =="Empty URL, Nothing found") or (content_syn =="Something went wrong with url"):
            
                    df_MS.at[index, 'Mention Content'] = row['Mention Content']
        else:
                    df_MS.at[index, 'Mention Content'] = content_syn
        
fin = time.time()
print(fin-inicio)


NameError: name 'df_MS' is not defined

In [ ]:
#Export 
df_MS.to_excel("Nombre.xlsx", engine='xlsxwriter') 
